In [11]:
import requests
import pandas as pd
import defs
from datetime import datetime
import pytz

Debo obtener la data de ESTA semana. Parametros en RFC3339.  
Considerando que solo obtengo 500 velas por llamada de la API y que cada dia tengo 1440 velasde M1, tengo que hacer 3 llamadas de la API por cada dia de la semana.
- Inicio: 2024-06-24T04:00:00.000000000Z  #hora de NY UTC-4
- Inicio: 2024-06-24T04:00:00.000000000Z

In [19]:
# Definir la zona horaria de Nueva York
ny_timezone = pytz.timezone('America/New_York')

# Obtener la fecha y hora actual en la zona horaria de Nueva York
time_ny = datetime.now(ny_timezone)
time_ny

datetime.datetime(2024, 6, 25, 2, 19, 16, 168211, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>)

In [21]:
import pytz
from datetime import datetime

def obtener_hora_nueva_york_rfc3339():
    # Definir la zona horaria de Nueva York
    zona_horaria_ny = pytz.timezone('America/New_York')

    # Obtener la fecha y hora actual en la zona horaria de Nueva York
    ahora_ny = datetime.now(zona_horaria_ny)

    # Formatear la fecha y hora en formato RFC3339
    hora_rfc3339 = ahora_ny.strftime("%Y-%m-%dT%H:%M:%S%z")

    return hora_rfc3339

# Ejemplo de uso
hora_actual_ny = obtener_hora_nueva_york_rfc3339()
print("Hora actual en Nueva York (RFC3339):", hora_actual_ny)

Hora actual en Nueva York (RFC3339): 2024-06-25T02:35:11-0400
